In [1]:
from numba import cuda
import networkx as nx
from itertools import combinations, groupby
import random
import time
from tqdm import tqdm

import sys
sys.path.append('..')

import graphcuda

cuda.gpus

c:\Users\Anderson\miniconda3\envs\gnn\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Pre-compile for [CuPy's JIT](https://docs.cupy.dev/en/stable/user_guide/performance.html#context-initialization)

In [2]:
graphcuda.generate_random_graph(1, 1)
graphcuda.generate_random_adjacency_matrix(1, 1)

array([[0.]])

### Parameters

In [3]:
n_nodes   = 10000
prob_edge = 1.2/(100-1)

# Test runs
K = 5

### Test original NetworkX approach

In [4]:
def gnp_random_connected_graph(n, p):
    """
    Generates a random undirected graph, similarly to an Erdős-Rényi 
    graph, but enforcing that the resulting graph is conneted
    """
    edges = combinations(range(n), 2)
    G = nx.Graph()
    G.add_nodes_from(range(n))
    if p <= 0:
        return G
    if p >= 1:
        return nx.complete_graph(n, create_using=G)
    for _, node_edges in groupby(edges, key=lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = random.choice(node_edges)
        G.add_edge(*random_edge)
        for e in node_edges:
            if random.random() < p:
                G.add_edge(*e)
    for (u, v) in G.edges():
        G.edges[u,v]['weight'] = random.random()*100
    return G

In [5]:
avg_time = 0
for _ in tqdm(range(K)):
    start = time.perf_counter()
    G = gnp_random_connected_graph(n_nodes, prob_edge)
    end = time.perf_counter()
    avg_time += end - start
avg_time /= K

print(f'NetworkX Generation Average Time: {avg_time:.03f} secs')

100%|██████████| 5/5 [01:12<00:00, 14.58s/it]

NetworkX Generation Average Time: 14.581 secs


### Test original Matrix Masking + CuPy approach

In [6]:
avg_time = 0
for _ in tqdm(range(K)):
    start = time.perf_counter()
    G = graphcuda.generate_random_graph(n_nodes, prob_edge)
    end = time.perf_counter()
    avg_time += end - start
avg_time /= K

print(f'Masking-CuPy Generation Average Time: {avg_time:.03f} secs')

100%|██████████| 5/5 [00:26<00:00,  5.30s/it]

Masking-CuPy Generation Average Time: 5.296 secs


### (Adjacency Matrix Only) Test original Matrix Masking + CuPy approach

In [9]:
avg_time = 0
for _ in tqdm(range(K)):
    start = time.perf_counter()
    G = graphcuda.generate_random_adjacency_matrix(n_nodes, prob_edge)
    end = time.perf_counter()
    avg_time += end - start
avg_time /= K

print(f'(Adjacency) Masking-CuPy Generation Average Time: {avg_time:.03f} secs')

100%|██████████| 5/5 [00:02<00:00,  1.71it/s]

(Adjacency) Masking-CuPy Generation Average Time: 0.583 secs
